In [2]:
import ROOT
import numpy as np
import time
import math
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.22/08


### Constants

In [3]:
nr_of_channels = int(3.6 * 1e6)
channels_per_module = int(15*30)
total_modules = int(nr_of_channels/channels_per_module)
global_counter = 0

### Noise Generator Class

In [4]:
class noise_generator:
    def __init__(self, threshold, channel_count): 
        global global_counter
        low = 0
        high = 1000
        noise = ROOT.TF1("noise_gauss", "TMath::Gaus(x, 0, 5)", low, high)
        
        """Why divide with 2?? Why do we need a avg. value?"""
        #integrate normal dist. to get exact prob. above threshold and multiply with channel nr to get avg.
        self._mean = channel_count * noise.Integral(threshold, high)/noise.Integral(low, high) 
        #self._prng = ROOT.TRandom3(time.time_ns())
        global_counter += 1
        self._prng = ROOT.TRandom3(int(time.time()) + global_counter)

    def generate(self):
        """The first return value is the exact number of noise activated channels in the small window and
            the second return value is the estimated number of noise activated channels in the small window. (estimated from large window)
            (both results are in small window, both results are noise above threshold)
            noise_exact, noise_est = noise.generate()"""
        # Large time window 25ns
        # Small time window 3.125ns
        prng = self._prng
        total = prng.Poisson(self._mean) # number of noise above threshold in large window in this run.
        narrow_exact = 0
        narrow_exact = (np.random.rand(total) < 0.125).sum()  # 3.125/25 = 0.125
        
        noise_per_time = (total - narrow_exact)/(25 - 3.125)
        estimate = 3.125 * noise_per_time
        return narrow_exact, estimate




 ### Signal Generator Class
 


In [5]:
class signal_generator:
    def __init__(self, threshold, real_expected_voltage, guess_expected_voltage): # expected_voltage = expected peak voltage  (mu)
        global global_counter
        low = -100
        high = 1000
        self._threshold = threshold
        self._expected_voltage = real_expected_voltage
        signal_func = ROOT.TF1("sth", "TMath::Landau(x, %f, %f)" % (guess_expected_voltage, guess_expected_voltage/4), low, high)
        self.fraction_above = signal_func.Integral(threshold, high)/signal_func.Integral(low, high) #fraction of the function that is above threshold
        #self._prng = ROOT.TRandom3(time.time_ns())
        global_counter += 1
        self._prng = ROOT.TRandom3(int(time.time()) + global_counter)
    
    def generate(self, expected_count):
        """The paramater is the expected number of particles that reach the detector.
            The first return value is the number of particles that reached the detector and
            the second return value is the number of activated channels.
            signal_real_total, signal_above = signal.generate(30)"""
        prng = self._prng
        expected_voltage = self._expected_voltage
        threshold = self._threshold
        
        signal_count = prng.Poisson(expected_count)
        above_threshold_count = 0
        sigma = expected_voltage/4
        for i in range(signal_count):
            if prng.Landau(expected_voltage, sigma) > threshold: # 1/4 is just the refernce 25/100
                above_threshold_count += 1
        return signal_count, above_threshold_count
    

## Simulation

In [46]:
"""Parameters for the simulation"""
threshold = 20 #mV
peak_mpv_voltage = 100 #mV, mpv: most probable value
guess_mpv = 100 #mV
expected_count = 15*1e3 #expected number of particles reaching the HGTD for mu=200, no more than 20k
#exp number of particles on detector proportional to mu
bunch_crossings = 10
ratio_of_channels_to_sample = 1
signal_to_channel_count_ratio = 0.5
"""Difference between mean and guess? How can we use guess estimated voltage?"""



"""For a number of BC, we get two lists: total_actual_signal_count and total_estimated_signal_count for each BC"""

channel_count = nr_of_channels*ratio_of_channels_to_sample
#avg_particles_reaching_detector = channel_count * signal_to_channel_count_ratio

avg_particles_reaching_detector = expected_count #15000 for mu=200
#generate the noise
noise_gen = noise_generator(threshold, channel_count)
#generate the signal
signal_gen = signal_generator(threshold, real_signal_mu, guess_signal_mu)

result_actual_signal_count = np.zeros(bunch_crossings)
result_estimated_signal_count = np.zeros(bunch_crossings)
error_estimation = 0
signal_fraction_above = signal_gen.fraction_above

for BC in range(bunch_crossings):
    # both results are in small window, both are noise above threshold
    noise_exact, noise_est = noise_gen.generate()

    # The paramater is the expected number of particles that reach the detector.
    # The first return value is the number of particles that reached the detector and 
    # the second return value is the number of activated channels.
    total_actual_signal, signal_above = signal_gen.generate(avg_particles_reaching_detector)

    # from the total activations, estimate how much was produced by actual signal
    total_activations = signal_above + noise_exact  # total activations in small window (i.e. above threshold)
    print("\nBunch Crossing %d:" %(BC+1))
    print("Signal- and noise-activated channels: %d" %(total_activations))
    print ("Noise-activated channels: %d" %(noise_exact))
    #total real activations minus estimation of signal, then divide with the prob. mass for Landau to be above threshold
    total_signal_est = (total_activations - noise_est)/signal_fraction_above 
    
    # assign the results
    result_actual_signal_count[BC] = total_actual_signal #exact count in the small window
    result_estimated_signal_count[BC] = total_signal_est #estimated count 
    print("True number of signal-activated channels: %d" %result_actual_signal_count[BC])
    print("Estimated number of signal-activated channels: %f" % result_estimated_signal_count[BC])
    error_estimation = 0
    


Bunch Crossing 1:
Signal- and noise-activated channels: 15012
Noise-activated channels: 21
True number of signal-activated channels: 14991
Estimated number of signal-activated channels: 14997.664842

Bunch Crossing 2:
Signal- and noise-activated channels: 14986
Noise-activated channels: 16
True number of signal-activated channels: 14970
Estimated number of signal-activated channels: 14971.950151

Bunch Crossing 3:
Signal- and noise-activated channels: 14838
Noise-activated channels: 15
True number of signal-activated channels: 14823
Estimated number of signal-activated channels: 14823.233523

Bunch Crossing 4:
Signal- and noise-activated channels: 14949
Noise-activated channels: 13
True number of signal-activated channels: 14936
Estimated number of signal-activated channels: 14933.235256

Bunch Crossing 5:
Signal- and noise-activated channels: 14976
Noise-activated channels: 14
True number of signal-activated channels: 14962
Estimated number of signal-activated channels: 14961.092837


### Study which threshold is best: with SNR or the signal deviation from true values? 


In [ ]:
"""Ratio between real signal and real noise? Or between estimated signal and noise?"""

"""Get the total deviation from the total simulated signals vs the expected signals.
        Returns both the total deviation as a positive number"""
real_signal_count = result_actual_signal_count.sum()
if real_signal_count == 0:
    total_deviation = 0
total_deviation = abs( result_estimated_signal_count.sum() / real_signal_count -1)

snr = 20 in the beginning
snr = 5 later 

large time window multiple of 3.125

todo later: check non-linear approximation of nr of particles vs mu 

